In [2]:
import pandas as pd
import numpy as np

In [3]:
def load_txt(path):
    with open(path,'r', encoding='utf-8') as file:
        linhas = file.readlines()
        lines = []
        for linha in linhas:
            linha = linha.strip()
            if linha != '':
                lines.append(linha)
    return lines

def create_df(column, lines):
    df_created = pd.DataFrame({column: lines})
    return df_created

def concat_df(list_df):
    df_concated = pd.concat(list_df, axis=1)
    return df_concated

In [4]:
import os

diretorio = r'..\database\problemas txt'

list_df = []
for nome_arquivo in os.listdir(diretorio):
    caminho_completo = os.path.join(diretorio, nome_arquivo)
    column = nome_arquivo[:-4]
    lines = load_txt(caminho_completo)
    df = create_df(column,lines)
    list_df.append(df)
    
new_df = concat_df(list_df)


In [5]:
new_df.head()

,arquivo1,arquivo10,arquivo100,arquivo101,arquivo102,arquivo103,arquivo104,arquivo105,arquivo106,arquivo107,...,arquivo90,arquivo91,arquivo92,arquivo93,arquivo94,arquivo95,arquivo96,arquivo97,arquivo98,arquivo99
0,A bomba está superaquecendo.,Consumo de energia elevado na bomba,Ruído agudo e constante vindo do motor elétrico,Ruído de clique regular no motor durante a par...,Barulho de batidas rápidas no motor elétrico,Problema de escova chatter em motores de corre...,Motor elétrico superaquecido,Comutador superaquecido em motores elétricos,Contatos elétricos com soldagem,Contatos elétricos desgastam rapidamente,...,Ruído excessivo durante o funcionamento do mot...,Rolamentos de bucha aquecendo além do normal,Rolamento superaquecido no motor elétrico,Problema de sobrecarga no controlador de motor...,Faíscas nas escovas de motores elétricos,Comutador escurecido em motores elétricos,Faíscas intermitentes nas escovas de motores e...,Poeira fina acumulada no acoplamento com amort...,Fumaça saindo do motor elétrico,Motor elétrico não liga
1,A bomba apresenta temperatura elevada.,Problema de alto consumo de energia na bomba,Zumbido intenso durante a operação do motor,Clique constante no motor elétrico,Vibração intensa e batidas rápidas no motor,Chatter excessivo nas escovas do motor elétrico,Motor esquentando demais,Problema de superaquecimento no comutador de m...,Problema de soldagem nos contatos,Vida útil reduzida dos contatos elétricos,...,Operação barulhenta em motores elétricos,Problema de superaquecimento nos rolamentos de...,Aquecimento excessivo do rolamento antifricção,Controlador apresentando sobrecarga excessiva ...,Flashes excessivos nas escovas de motores,Problema de escurecimento do comutador,Problema de faíscas ocasionais nas escovas de ...,Problema de sujeira sob o acoplamento com pino...,Bigodes de solda na armadura do motor,Falha na partida do motor
2,A bomba está esquentando demais.,Bomba consumindo mais energia do que o normal,Barulho irritante de zumbido no motor principal,Barulho de clique em intervalos regulares no m...,Ruído de batidas rápidas em motores elétricos,Escovas vibrando de forma irregular no motor,Superaquecimento excessivo do motor,Comutador aquecendo demais em motores elétricos,Soldagem indesejada dos contatos elétricos,Problema de curta duração dos contatos elétricos,...,Motor elétrico produzindo ruídos estranhos,Rolamentos de bucha operando em alta temperatura,Problema de temperatura elevada no rolamento d...,Sobrecarga comprometendo o desempenho do contr...,Problema de faíscas em motores elétricos,Comutador escuro em motores,Ocorrência de faíscas intermitentes em escovas...,Acoplamento com amortecedores ou pinos de borr...,Solda derretida formando filamentos no motor,Problemas de partida no motor elétrico
3,O motor da bomba está superaquecido.,Consumo excessivo de energia no funcionamento ...,Excesso de ruído vibrante no motor elétrico,Ruído de clique rítmico no motor principal,Problema de batidas rápidas no motor principal,Ruído anormal nas escovas do motor durante a o...,Temperatura elevada no motor elétrico,Aquecimento excessivo do comutador em motores,Contatos elétricos que ficam grudados por solda,Contatos elétricos não duram muito tempo,...,Problema de barulho anormal na operação do mot...,Aquecimento excessivo nos rolamentos de bucha,Rolamento antifricção aquecendo além do normal,Controlador enfrentando problemas de sobrecarg...,Escovas produzindo faíscas excessivas,Escurecimento anormal do comutador em motores ...,Faíscas intermitentes afetando o desempenho da...,Acoplamento com amortecedores ou pinos de borr...,Problema de fumaça e solda na armadura,Motor não inicia
4,A bomba está piscando e emitindo calor excessivo.,Problema de eficiência energética na bomba,Zumbido perturbador proveniente do motor de co...,Motor elétrico emitindo cliques regulares,Batidas constantes e rápidas no motor elétrico,Escovas saltando ou tremendo no motor elétrico,Problema de superaquecimento no motor,Motor

In [9]:
import random

min_size = min([len(new_df[column].dropna()) for column in new_df.columns])

def undersample_dataframe(df, min_size, random_seed=None):
    random.seed(random_seed)
    undersampled_dfs = []

    for column in df.columns:
        column_data = df[column].dropna().tolist()
        undersampled_column = random.sample(column_data, min_size)
        undersampled_dfs.append(undersampled_column)

    undersampled_df = pd.DataFrame(undersampled_dfs).T
    undersampled_df.columns = df.columns

    return undersampled_df

undersampled_df = undersample_dataframe(new_df, min_size)
undersampled_df.shape

50


(50, 116)

In [8]:
undersampled_df.to_excel(r'..\database\problems\undersampling_test.xlsx')

In [129]:
from sklearn.model_selection import train_test_split

def prepare_data(df, test_size=0.3, random_state=None):
    classes = df.columns.tolist()

    X = []
    y = []

    for _, row in df.iterrows():
        X.append(row.values)
        y.append(classes)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test


In [130]:
X_train, X_test, y_train, y_test = prepare_data(new_df)